In [1]:
#math and linear algebra stuff
import numpy as np

#Math and linear algebra stuff
import scipy.stats as scs

#plots
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16.0, 9.0)
#mpl.rc('text', usetex = True)
import matplotlib.pyplot as plt
%matplotlib inline

# Normalizing flows for Deep Learning

## Introduction

This notebook is somehow a follow up the InformationTheoryOptimization notebook. Many more basic concepts exposed there will be used here. But we will give a short reminder.

## Definitions

### Bayes theorem

Although Bayesian statistics is not really a novel approach, the recent rise of probabilistic programming libraries, availability of a lot of data, and powerful computer to run markov chains, made the tedious task of running bayesian inference algorithms a lot easier.

Let's recall some simple elements about the Bayes theorem that we have been in other notebooks:

For two given random variables, $X$ and $Y$, we can write :
\begin{equation}
    P(\theta|x) = \frac{P(x|\theta)P(\theta)}{P(x)}
\end{equation}


We could use a slightly more formal version, that includes a model $\mathcal{M}$ that we believe is the underlying model for the random variable $x$, parametrized by $\theta$

\begin{equation}
    P(\theta|x,\mathcal{M}) = \frac{P(x|\theta, \mathcal{M})P(\theta|\mathcal{M})}{P(x|\mathcal{M})}
\end{equation}

* Usually, $\theta$ is the random variable that we would like to caracterize. It usually consist in a set of parameters for a given statistical model (mean and variance for a normal distribution for instance).

* On the contrary $x$, called the evidence, is usually derived from a known dataset, or from sampling a real life process. We will see shortly how we can define an empirical distribution from a set of samples. 

* We call $P(x|\theta)$ the likelihood of $x$, given the model distribution  $\theta$. This is the likelihood (or its logarithm) we tried to maximize with various instances of expectation maximization algorithm in the InformationTheory notebook by finding the optimal model parameters $\theta$.
Maximum likelihood had some success in the past for some instance of problems where likelihood or its logarithm were concave and differentiable, and a gradient based methods allowed to find a global maximum.

* We call $P(\theta|x)$ the posterior distribution of $\theta$, given a known (usually empirical distribution of data) $x$. As opposite to the prior, it gives an idea of the probability of the model AFTER some data ($x$) has been seen.

* We call $P(\theta)$ the a-priori distribution for the variable $\theta$. This one can be derived if we have a-priori knowledge on the model parameters $\theta$, it may consist in apriori knowledge of some surrogate parameters that we integrate as a marginal distribution and scale. A prior usually allows us to compute the probability of a given set of parameters $\theta \in \mathbb{R}^n$ that feed the model $\mathcal{M}$.

A priori usually writes:
  \begin{align*}
    y &\rightarrow P(\theta=y) \\
    \mathbb{R}^n &\mapsto [0,1]
  \end{align*}
  
A concrete example is for instance, the use of the framework of random markov field (or Gibbs random field) in a n-dimensional space like an image, were we can consider the pixels as a set of vertices of a graph, and only neighbouring pixels are connected by edges. The probability of a given graph $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ is given by $P(\mathcal{G}) = \alpha e^{-\beta U(\mathcal{G})}$ where $\alpha$ and $\beta$ are normalization factor, and $U(\mathcal{G}) = \sum_{c\in C} V_c(\mathcal(G))$ is a sum of clique potentials $V_c(\mathcal(G))$ over all possible cliques $C$. A very common instance of Gibbs measure is the gaussian-like distribution where, we have, for each neighbouring pair of pixel $p_1-p_2$ (a clique): $V_{p_1-p_2}(\mathcal{G}) = (\mathcal{G}_{p_1}-\mathcal{G}_{p_2})^2$
When one has no apriori on $\theta$, then the highest entropy hypothesis is implicitly used. More pragmatically, we write $P(\theta=y) \sim Uniform(\mathrm{supp} \theta)$ where $\mathrm{supp} y)$ is the support of $\theta$
Notice that, given a discrete process, as long as we received new data, the prior distribution can be taken just as being the previous step posterior.

* Usually $P(x)$ is the empirical distribution that correspond to a given set of outcomes (actual data). It represents the probability of the given outcome of the experiment regardless of the value of the underlying model $\theta$. We can see $P(x)$ as a marginalization of $\theta$ in the joint distribution $P(x,\theta)=P(x|\theta)P(\theta)$ i.e. $P(x) = \int_{\theta} P(x,\theta)$ such that it acts as a constant used to normalize likelihood  / prior product to make sure the result posterior is a distribution.

### Bayesian inference and the sampling task

As briefly seen in the InformationTheory notebook, in the general case, to compute the posterior over the whole support of $\theta$ is extremely complex problem, especially if $\theta$ is a vector from a high dimension space, and the actual distribution of the posterior is very complex (not smooth, many local dense areas, etc...). MCMC types of methods often comes into play here, where one computes $P(\theta=y_0|x)$ for a given $y_0$, then jumps to another point in space $y_1$, with a given probably, dependant on $P(\theta=y|x)$.
Little by little one can get a discrete approximation of the posterior, but it can be very slow.

## Kullback Leibler divergence

The Kullback Leibler divergence, also called the relative entropy or discrimination information, can be computed between two distributions, say $x$ and $y$ and writes $ D_{KL}(x\|y) $.
This metric represents the expectation of the difference of the number of bits needed to encode a symbol from the distribution x, weither the encoding is optimal for the distribution x or y.

The general case reads:

\begin{align}
    D_{KL}(x\|y) &= \mathbb{E}_x \left[ log \left( \frac{x}{y} \right) \right]
\end{align}

In the discrete case it reads:

\begin{align}
    D_{KL}(x\|y) &= \sum_{i\in I} p(x_i) log(p(x_i)) -\sum_{i\in I} p(x_i) log(p(y_i)) \\
                 &= -H(x) + H_y(x)
\end{align}
or equivalently
$$
    D_{KL}(x\|y) = \sum_{i\in I} p(x_i) log\left(\frac{p(x_i)}{p(y_i)}\right)
$$


### KL Divergence and maximum likelihood
This chapter was inspired by [this article](http://www.hongliangjie.com/2012/07/12/maximum-likelihood-as-minimize-kl-divergence/)
by Liangjie Hong

#### Definitions
Let's consider $P(x_i|\theta)$ a distribution used to generate (or sample) $N$ points $x_i, i=0,\dots,N-1$

We can define a general distribution model as:
$$
    P_\theta(x) = P(x|\theta)
$$
And, to handle data from real world, we will define the following empirical distribution (also called probability mass function):
$$
    P_D(x) = \sum_{i=0}^{N-1} \frac{1}{N} \delta(x-x_i)
$$

#### KL Divergence between model and data
We can now express the Kullback Leibler divergence between the model distribution and the empirical one:

\begin{align}
    D_{KL} \left( P_D(x)||P_\theta(x) \right) &= \int P_D(x) log(P_D(x)) dx -\int P_D(x) log(P_\theta(x)) dx \\
    &= H(P_D(x)) - \int P_D(x) log(P_\theta(x)) dx
\end{align}

As $H(P_D(x))$ is a constant term, relatd to the empirical distribution of real data, we will call it $\epsilon$, and we may get rid of it later.
Instead we will mainly consider, the $\theta$, ie, the model related term:
$$
   D_{KL} \left( P_D(x)||P_\theta(x) \right) = \epsilon -\langle P_D(x) , log(P_\theta(x)) \rangle_{L^2}
$$

Thanks to the definition of the empirical distribution, we can express this continuous expression as a discrete summation:

\begin{align}
    \langle P_D(x) , log(P_\theta(x)) \rangle_{L^2} &= \int \sum_{i=0}^{N-1} \frac{1}{N} \delta(x-x_i) log(P(x|\theta)) dx \\
    &= \sum_{i=0}^{N-1} \frac{1}{N} log(P(x_i|\theta)) \\
    &= \frac{1}{N} \sum_{i=0}^{N-1} log(P(x_i|\theta))
\end{align}

The last line amounts to the log-likelihood of the dataset. We can then conclude that maximizing the log-likelihood of a dataset, relatively to a set of distribution parameters $\theta$ and a given dataset $x$ is equivalent to minimizing the Kullback Leibler divergence between the empirical distribution and the model distribution.

### Practical KL Divergence between two data sets
We are interested in computing the Kullback Leibler divergence between a model $X$ and a real dataset $Y$, indexed by $i$ where $i$ is the index of a specific event whose occurence probability is $X_i$ according to the model, and $Y_i$ in practice.

As in the definition of the empirical distribution, our datasets must verify:
$$
    \sum_{i=0}^{N-1} X_i = 1
$$

then the expression of the kl divergence is simply:
$$
    D_{KL}(Y\|X) = \sum_{i=0}^{N-1} log \left( \frac{Y_i}{X_i} \right) Y_i
$$

Moreover there is an important condition that must be respected: 
$$
    X_i = 0 \implies Y_i = 0
$$
If so, if the event $k$ has no occurence, we have $X_k=Y_k=0$ and $log \left( \frac{y_k}{x_k} \right) y_k$ has a limit in $0$ which is $0$. Otherwise, Kullback Leibler cannot be computed.

The explanation is pretty straightforward: if an event, or a symbol from $Y$ has no occurence in $X$, it has no reason to be encoded using $X$ probability distribution, then the average number of bits needed to encode a symbol from $Y$ is a nonsense, as some symbols cannot be represented.

## What is a variational inference ?

We are still in the framework of bayes theorem, and we want to compute $p$ our posterior that stands for $p(\theta|x)$.

In variational inference,  you create a variational distribution $q$ over your model parameters $\theta$, that we will call latent variables here. A variational distribution is parametrized by a variational parameter, we will call $\nu$:

$$
 q(\theta|\nu)
$$

The first idea of variational inference, is that we will try to find $\nu$ such that $q(\theta|\nu)$ becomes close to our posterior distribution.
The closeness of those distribution will be computed using Kullbac-Liebler divergence:
$$
\begin{align}
    D_{KL}(q\|p) &= \mathbb{E}_q \left[ log \left( \frac{q(\theta)}{p(\theta|x)} \right) \right]
\end{align}
$$


## Normalizing flows: main references
Here are a few very important references we might refere to

In [2]:
from IPython.display import IFrame

Lets start with the amazing work of Mahdi Karami. The document is a PhD thesis dedicated to the study of tools for variational inference, normalizing flows, analysis and design of operator with tractable inverse jacobian computation, with additional numerical and convex optimization tools. It also contains applications to latent space representations of complex distributions met in signal/image processing, and much more:

In [6]:
IFrame("doc/NormalizingFlows/Karami_Mahdi_202008_PhD.pdf", width=1200, height=800)

Then lets move on to a very interesting review paper: Normalizing Flows: An Introduction and Review of Current Methods by Ivan Kobyzev, Simon J.D. Prince, and Marcus A. Brubaker.

In [7]:
IFrame("doc/NormalizingFlows/1908.09257.pdf", width=1200, height=800)

Another reference paper, that exposes a lot of important properties of Normalizing flows, and design of methods to leverage them in machine learning / representation learning: Normalizing Flows for Probabilistic Modeling and Inference by George Papamakarios, Eric Nalisnick, Danilo Jimenez Rezende, Shakir Mohamed, Balaji Lakshminarayanan

In [8]:
IFrame("doc/NormalizingFlows/1912.02762.pdf", width=1200, height=800)

Then a paper more targeted to application: Same Same But DifferNet: Semi-Supervised Defect Detection with Normalizing Flows by Marco Rudolph, Bastian Wandt and Bodo Rosenhahn:

In [10]:
IFrame("doc/NormalizingFlows/2008.12577.pdf", width=1200, height=800)